In [1]:
import pandas as pd
import numpy as np
from math import floor

df = pd.read_csv("CSIRO_Recons_gmsl_yr_2019.csv", header=0, index_col=0)
df.drop(columns=["GMSL uncertainty (mm)"], axis=1, inplace=True)
df.rename(columns={"Time": "year", "GMSL (mm)": "sea_level"}, inplace=True)

df.index = np.floor(df.index).astype(int)
df.index = pd.to_datetime(df.index, format="%Y").year
df.head()

,sea_level
Time,
1880,-30.3
1881,-24.7
1882,-41.5
1883,-36.2
1884,-15.3


In [2]:
# grid search ARIMA parameters for time series
import warnings
from math import sqrt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
 
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    # make predictions
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
    # calculate out of sample error
    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(dataset, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print('ARIMA%s RMSE=%.3f' % (order,rmse))
                except:
                    continue
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
    return best_cfg
 
# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
# best_cfg = evaluate_models(df.values, p_values, d_values, q_values)

c:\anaconda\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
c:\anaconda\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [6]:
# Rebuild ARIMA using best configuration
X = df.values
train_size = int(len(X) * 0.66)
train, test = X[0:train_size], X[train_size:]
history = [x for x in train]

predictions = list()
for t in range(len(test)):
  model = ARIMA(history, order=(1,2,1))
  model_fit = model.fit()
  yhat = model_fit.forecast()[0]
  predictions.append(yhat)
  history.append(test[t])

In [9]:
display(predictions)

[94.0286851349696,
 102.28203203516165,
 100.1351246734584,
 107.15744404012696,
 109.41395962684732,
 108.20048159822478,
 106.65095003447811,
 110.80437428711441,
 109.11091640045787,
 111.93794052346948,
 122.75160495171922,
 122.09259888923145,
 126.31496849002251,
 128.2271116261594,
 120.55668107547865,
 117.89451683712711,
 118.48488057941579,
 122.0577742181787,
 126.66261244409573,
 128.93367852156783,
 131.57608337492113,
 134.47687364349915,
 131.96344555218053,
 133.15886524225775,
 137.46183507321186,
 142.54953315296393,
 149.1473128230673,
 142.85502372835214,
 146.58500989797326,
 152.09392962468746,
 157.0921245095023,
 160.76269935645362,
 167.74729411067415,
 169.20742068791725,
 169.10791593970848,
 172.1780012794678,
 174.34645371522137,
 180.8463050427549,
 188.1126616750219,
 195.0201417622609,
 197.7197165338316,
 203.37179573832117,
 203.5315013840394,
 210.9686195085842,
 211.66833583647562,
 210.79766880033293,
 216.0656890084677,
 219.16436487616363]